In [9]:
#  Links consultados:
#  https://wiki.postgresql.org/wiki/Psycopg2_Tutorial
#  https://www.psycopg.org/docs/
#  https://www.psycopg.org/docs/usage.html#passing-parameters-to-sql-queries
#  https://stackoverflow.com/questions/18664074/getting-error-peer-authentication-failed-for-user-postgres-when-trying-to-ge

!pip install PySimpleGUI
!pip install psycopg2-binary

import PySimpleGUI as sg
import psycopg2

def limpar():
    window['-ID-'].update('')
    window['-NOME-'].update('')
    window['-EMAIL-'].update('')
    window['-CEL-'].update('')

def atualiza():
    if len(lista) == 0:
        limpar()
    else:
        window['-ID-'].update( lista[indice][0] )
        window['-NOME-'].update( lista[indice][1] )
        window['-EMAIL-'].update( lista[indice][2] )
        window['-CEL-'].update( lista[indice][3] )

def todos():
    global indice
    global lista
    resposta = []
    with con:
        with con.cursor() as cursor:
            cursor.execute("SELECT * FROM amigos;")
            resposta = cursor.fetchall()
    lista.clear()
    for i in range(len(resposta)):
        lista.append( list(resposta[i]) )

    sg.popup('Quantidade de registros: ' + str(len(resposta)))
    indice = 0
    atualiza()

# Inicialização BD
con = psycopg2.connect(host="localhost", database="starfit", user="postgres", password="123456")
with con:
    with con.cursor() as cursor:
        cursor.execute("""CREATE TABLE IF NOT EXISTS aluno (
            id_aluno SERIAL PRIMARY KEY UNIQUE NOT NULL,
            aluno_nome VARCHAR(50) NOT NULL
            );""")

lista=[]
indice = 0

layout = [
    [
        sg.Text("ID:", size=(8, 1)),
        sg.InputText(size=(6, 1), key="-ID-", focus=False)
    ],
    [
        sg.Text("Nome:", size=(8, 1)),
        sg.InputText(size=(40, 1), key="-NOME-", focus=True)
    ],
    [
        sg.Text("E-mail:", size=(8, 1)),
        sg.InputText(size=(40, 1), key="-EMAIL-")
    ],
    [
        sg.Text("Celular:", size=(8, 1)),
        sg.InputText(size=(40, 1), key="-CEL-")
    ],
    [
        sg.Button('Limpar', size=(8, 1), key="-LIMPAR-"),
        sg.Button('Inserir', size=(8, 1), key="-INSERIR-"),
        sg.Button('Atualizar', size=(8, 1), key="-ATUALIZAR-"),
        sg.Button('Remover', size=(8, 1), key="-REMOVER-")
    ],
    [
        sg.Button('<<', size=(8, 1), key="-<<-"),
        sg.Button('Procurar', size=(8, 1), key="-PROCURAR-"),
        sg.Button('Todos', size=(8, 1), key="-TODOS-"),
        sg.Button('>>', size=(8, 1), key="->>-")
    ]
]

window = sg.Window("Starfit", layout, use_default_focus=False)

# Run the Event Loop
while True:
    event, values = window.read()

    if event == sg.WIN_CLOSED:
        break
    elif event == "-LIMPAR-":
        limpar()
    elif event == "-INSERIR-":
        with con:
            with con.cursor() as cursor:
                cursor.execute("INSERT INTO amigos (nome, email, celular) VALUES (%s, %s, %s);",
                    (values['-NOME-'], values['-EMAIL-'], values['-CEL-']))
        limpar()
    elif event == "-ATUALIZAR-":
        with con:
            with con.cursor() as cursor:
                cursor.execute("UPDATE amigos SET nome = %s, email = %s, celular = %s WHERE id = %s",
                    (values['-NOME-'], values['-EMAIL-'], values['-CEL-'], values['-ID-']))
        lista[indice] = [values['-ID-'], values['-NOME-'], values['-EMAIL-'], values['-CEL-']]        
    elif event == "-REMOVER-":
        with con:
            with con.cursor() as cursor:
                cursor.execute("DELETE FROM amigos WHERE id = %s", (values['-ID-'],))
        lista.pop(indice)
        indice -= 1
        atualiza()
    elif event == "-PROCURAR-":
        with con:
            with con.cursor() as cursor:
                cursor.execute("SELECT * FROM amigos WHERE nome LIKE %s;",
                    ('%'+values['-NOME-']+'%',))
                resposta = cursor.fetchall()
                lista.clear()
                for i in range(len(resposta)):
                    lista.append( list(resposta[i]) )
                sg.popup('Quantidade de registros: ' + str(len(resposta)))
                indice = 0
                atualiza()
    elif event == "-TODOS-":
        todos()
    elif event == "->>-":
        indice += 1
        if indice >= len(lista): indice = len(lista)-1
        atualiza()
    elif event == "-<<-":
        indice -= 1
        if indice <= 0: indice = 0
        atualiza()

window.close()

# Fazer as mudanças para a base persistente
con.commit()

# Fechar a comunicação com o servidor
cursor.close()
con.close()


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
